https://jira.x5.ru/browse/CVMXC-2675

In [1]:
name = 'CVMXC-2675_bonduelle'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *
from logic_filters import *
from segmentation import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/24 08:58:30 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Выбираем гостей нужного юзкейса

In [5]:
usecase = ['upgrade', 'cross', 'ump']
dt = datetime.date.today()
lifetime = 90
freq = 1

In [6]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
                     .filter(F.col('lifetime') >= lifetime)
                     .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )


In [7]:
print(customers_usecase.count()) #16775430

[Stage 0:====================================================>     (9 + 1) / 10]

15868350


### 1 проверяем на доступность отобранных гостей в определенную дату

In [8]:
check_date = '2022-03-07'

In [9]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:16


In [10]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:07


In [11]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:15


In [12]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [13]:
# seg_sms.write.parquet('temp2675_1', mode='overwrite')
seg_sms = spark.read.parquet('temp2675_1')

In [14]:
seg_sms.count() #

7597146

In [15]:
check_date = '2022-03-14'

In [16]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [17]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:02


In [18]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:11


In [19]:
seg_sms_2 = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [20]:
seg_sms_2.write.parquet('temp2675_2', mode='overwrite')
seg_sms_2 = spark.read.parquet('temp2675_2')

22/02/18 13:51:49 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.58:50840 is closed


In [21]:
seg_sms_2.count() #

7603152

### Проверка на доступность плю в магазинах

ЦФО

In [22]:
plu_codes = [20062, 20063, 39419, 39420, 4038156, 4038159]

In [23]:
macroregion_dk = ['MRDCN']

In [24]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
            .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

2991

### 1

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [25]:
start_date = datetime.date(2021, 11, 10)
end_date = datetime.date(2022, 2, 9)

In [26]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [27]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [28]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [29]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [30]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [31]:
plu_hierarchy_lvl_4_dk = ['FD0205011']
# syntethic_category_id = [53]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [32]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [33]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [34]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [35]:
accs = ['account_no']
pdf = checks_tc5.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc5 = checks_tc5.join(pdf, on=accs)

In [36]:
seg1 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [37]:
seg1.write.parquet('temp2675_seg1', mode='overwrite')
seg1 = spark.read.parquet('temp2675_seg1')

In [38]:
seg_pd1 = seg1.toPandas()

In [39]:
seg_pd1.head()

,customer_rk
0,12064208
1,27079759
2,13486269
3,16989155
4,16367857


In [40]:
seg_no = 'seg1'

In [41]:
seg_pd1.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [42]:
seg_pd1['customer_rk'] = seg_pd1['customer_rk'].astype('str')

In [43]:
cross, upgrade, ump = get_usecases(seg_pd1)

In [44]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 72073, upgrade: 93431, ump: 29391 (all: 194895)


### Проверка на доступность плю в магазинах

СПБ, СИБИРЬ, УРАЛ, МОСКВА, ВОЛГА

In [45]:
plu_codes = [4087615, 4087616]

In [46]:
macroregion_dk = ['MRDSB', 'MRDUR', 'MRDVO']
federal_subject_dk = [77, 78]

In [47]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops1 = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
            .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            )
shops2 = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
#             .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            )
shops = shops1.union(shops2).distinct()


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

7226

### 2

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [65]:
start_date = datetime.date(2021, 11, 10)
end_date = datetime.date(2022, 2, 9)

In [66]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [67]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [68]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms_2, on='customer_rk', how='inner')

In [69]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [70]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [71]:
# plu_hierarchy_lvl_4_dk = ['FD0205011']
syntethic_category_id = [34, 35, 36, 37]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [72]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [73]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [74]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [75]:
accs = ['account_no']
pdf = checks_tc5.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc5 = checks_tc5.join(pdf, on=accs)

In [76]:
seg2 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [77]:
seg2.write.parquet('temp2675_seg2', mode='overwrite')
seg2 = spark.read.parquet('temp2675_seg2')

In [78]:
seg_pd2 = seg2.toPandas()

In [79]:
seg_pd2.head()

,customer_rk
0,11452643
1,12579676
2,21433200
3,18762408
4,14174589


In [80]:
seg_no = 'seg2'

In [81]:
seg_pd2.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [82]:
seg_pd2['customer_rk'] = seg_pd2['customer_rk'].astype('str')

In [83]:
cross, upgrade, ump = get_usecases(seg_pd2)

In [84]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 823633, upgrade: 970264, ump: 395783 (all: 2189680)


### 3

### Выбираем гостей нужного юзкейса

In [6]:
usecase = ['upgrade', 'cross', 'ump']
dt = datetime.date.today()
lifetime = 90
freq = 1

In [7]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
                     .filter(F.col('lifetime') >= lifetime)
                     .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )


In [8]:
print(customers_usecase.count()) #16775430

[Stage 1:>                                                          (0 + 1) / 1]

0


### 1 проверяем на доступность отобранных гостей в определенную дату

In [9]:
check_date = '2022-03-14'

In [9]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:08


In [10]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:07


In [11]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:06


In [12]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [10]:
# seg_sms.write.parquet('temp2675_3', mode='overwrite')
seg_sms = spark.read.parquet('temp2675_3')

In [11]:
seg_sms.count() #

7597146

### Проверка на доступность плю в магазинах

РФ

In [12]:
plu_codes = [20063]

In [13]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
#             .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

22245

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [14]:
start_date1 = datetime.date(2021, 10, 22)
end_date1 = datetime.date(2021, 12, 21)
start_date2 = datetime.date(2021, 12, 22)
end_date2 = datetime.date(2022, 2, 21)

In [15]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [16]:
checks_headers = checks_headers.filter(F.col('store_id').isin(plants))

In [17]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [18]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [19]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [20]:
plu_codes_cat = [39420, 20060, 21308, 20062, 4038159, 3406692, 4038156, 3500611, 39419, 20063, 4038159]

In [21]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [22]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [23]:
seg1 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [24]:
checks_headers2 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [25]:
checks_headers2 = checks_headers2.filter(F.col('store_id').isin(plants))

In [26]:
loyalty_cards2 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders2 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info2 = loyalty_cards2.join(loyalty_cardholders2, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms, on='customer_rk', how='inner')

In [27]:
checks_headers_tc52 = checks_headers2.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [28]:
checks_items2 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [29]:
checks_items2 = checks_items2.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [30]:
checks_tc52 = checks_items2.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [31]:
seg2 = (checks_tc52.select('customer_rk')
                  .distinct()
      )

In [32]:
seg_no_cheese_1 = seg_sms.join(seg2, 'customer_rk', 'left_anti')

In [33]:
seg = seg_no_cheese_1.join(seg1, 'customer_rk', 'inner')

In [34]:
seg.write.parquet('temp2675_ottok_new', mode='overwrite')
seg = spark.read.parquet('temp2675_ottok_new')

In [35]:
seg_pd1 = seg.toPandas()

In [36]:
seg_pd1.head()

,customer_rk
0,11432414
1,11557547
2,11807090
3,11862035
4,11907012


In [37]:
seg_no = 'seg3'

In [38]:
seg_pd1.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [39]:
seg_pd1['customer_rk'] = seg_pd1['customer_rk'].astype('str')

In [40]:
cross, upgrade, ump = get_usecases(seg_pd1)

In [41]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 28278, upgrade: 35441, ump: 13549 (all: 77268)
